In [1]:
import open3d as o3d
import numpy as np
from open3d.j_visualizer import JVisualizer

In [2]:
import os
import os.path
import shutil

import numpy as np
from torchsparse import SparseTensor
from torchsparse.utils.collate import sparse_collate_fn
from torchsparse.utils.quantize import sparse_quantize

from sklearn.neighbors import KDTree

In [18]:
class ErasorCarlaInternal:

    def __init__(self,
                 root,
                 voxel_size,
                 num_points,
                 visualize,
                 split,
                 proposal=False,
                 sample_stride=1,
                 submit=False,
                 google_mode=True,
                 window=10,
                 radius=50,
                 neighbor_k=0,
                 thres=0.02,
                 save_npy=False,
                 index=None):
        
        self.root = root
        self.split = split
        self.proposal = proposal
        self.submit = submit
        self.google_mode = google_mode
        self.visualize = visualize
        self.save_npy = save_npy # only in jupyter
        self.index = index # only in jupyter

        self.voxel_size = voxel_size
        self.num_points = num_points
        self.sample_stride = sample_stride
        self.window = window
        self.radius = radius
        self.neighbor_k = neighbor_k
        self.thres = thres
        
        self.seqs = []
        
        if split == 'train':
            self.seqs = [
               'scenario2', 'scenario3', 'scenario5', 'scenario6', 'scenario8'
            ]

        elif self.split == 'val':
            self.seqs = [
               'scenario2', 'scenario3', 'scenario5', 'scenario6', 'scenario8'
            ]

        elif self.split == 'test':
            self.seqs = [
               'scenario2', 'scenario3', 'scenario5', 'scenario6', 'scenario8'
            ]

        self.map_files = dict()
        self.scan_files = []

        # get scan and map data list
        for seq in self.seqs:
            self.map_files[seq] = os.path.join(self.root, 'testing_map/v0.1', seq, 'map.npy')
            seq_files = sorted(os.listdir(os.path.join(self.root, 'testing_data', seq, 'global_npz')))
            seq_files = seq_files[(self.window + 1) // 2 - 1: - (1 + self.window // 2) ]
            seq_files = [os.path.join(self.root, 'testing_data', seq, 'global_npz', x) for x in seq_files]
            self.scan_files.extend(seq_files)

        # get map_data
        self.map = dict()
        for seq, map_file in self.map_files.items():
            map_ = np.load(map_file)
            # get pesudo label with ERASOR
            """
            def get_pseudo_label(self, map_, block_multi, **kwargs):
                # return dynamic and static labels of map
                return map_[:, 3]
            pseudo_label = self.get_pseudo_label(map_, block_multi, **kwargs)
            """
            self.map[seq] = map_.astype(np.float32)

        if self.sample_stride > 1:
            self.scan_files = self.scan_files[::self.sample_stride]
    

        self.num_classes = 2
        self.angle = 0.0
    
    def revise_dynamic_points(self, points, labels, thres):
        labels[(labels != 0) & (points[:, 2] < thres)] = 0
        return labels
    
    def concatenate_scans(self, index):
        scan = np.load(self.scan_files[index])
        odom = scan['arr_1'].astype(np.float32)

        file_name = os.path.basename(self.scan_files[index])
        file_dir = os.path.dirname(self.scan_files[index])
        file_name_wo_ext = os.path.splitext(file_name)[0]
        file_int = int(file_name_wo_ext)
        file_int_list = list(range(file_int - (self.window + 1) // 2 + 1, file_int + self.window // 2 + 1, 1))
        file_str_list = [file_dir + "/" + str(i).zfill(6) + ".npz" for i in file_int_list]
        block_ = []
        for file in file_str_list:
            scan_ = np.load(file)
            block_single = scan_['arr_0'].astype(np.float32)
            block_.extend(block_single)
        block_ = np.asarray(block_)
        # radius search w.r.t the odom of scan data
#         block_ = block_[np.sum(np.square(block_[:, :2] - odom[:2]), axis=-1) < self.radius*self.radius]
        block_ = block_[np.sum(np.square(block_[:, :3] - odom), axis=-1) < self.radius*self.radius]
        
        print("radius; concatenated scan: ", np.shape(block_))
        
        return block_, odom
    

    def set_angle(self, angle):
        self.angle = angle

    def __len__(self):
        return len(self.scan_files)

    def __getitem__(self, index):
        
        if self.index is not None:
            index = self.index
        
        # get scan_data
        if self.window == 1:
            scan = np.load(self.scan_files[index])
            block_ = scan['arr_0'].astype(np.float32)
            odom = scan['arr_1'].astype(np.float32)
        else:
            # concatenate the scan data with window size
            block_, odom = self.concatenate_scans(index)
        
        # get map_data
        scenario = self.scan_files[index]
        scenario = scenario.split("/")[-3]
        map_ = self.map[scenario]
        print("orignal; map point len: ", np.shape(map_[:, 3]))
        print("orignal; map point element: ", np.unique(map_[:, 3]))
        
        scan = np.load(self.scan_files[index])
        odom_ = scan['arr_1'].astype(np.float32)
        print("odom: ", odom)
        print("odom_: ", odom_)
            
        
        # radius search w.r.t the odom of scan data
        map_ = map_[np.sum(np.square(map_[:, :3] - odom), axis=-1) < self.radius*self.radius]
#         map_ = map_[np.sum(np.square(map_[:, :2] - odom[:2]), axis=-1) < self.radius*self.radius]
        print("odom: ", odom)
        print("radius; map points len: ", np.shape(map_))
        print("radius; map point element: ", np.unique(map_[:, 3]))

                        
        # save npy
        if self.save_npy:
            self.save_npy(map_, odom)         
        
        block = np.zeros_like(block_)
        map = np.zeros_like(map_)
        

        if 'train' in self.split:
            # data augmentation on the train dataset: off in the ipynb
            theta = np.random.uniform(0, 2 * np.pi)
            scale_factor = np.random.uniform(0.95, 1.05)
            rot_mat = np.array([[np.cos(theta), np.sin(theta), 0],
                                [-np.sin(theta),
                                 np.cos(theta), 0], [0, 0, 1]])

            block[:, :3] = np.dot(block_[:, :3], rot_mat) * scale_factor
            map[:, :3] = np.dot(map_[:, :3], rot_mat) * scale_factor
#             theta = self.angle
#             transform_mat = np.array([[np.cos(theta),
#                                        np.sin(theta), 0],
#                                       [-np.sin(theta),
#                                        np.cos(theta), 0], [0, 0, 1]])
#             block[...] = block_[...]
#             map[:, :3] = map_[:, :3]

        else:
#             theta = self.angle
#             transform_mat = np.array([[np.cos(theta),
#                                        np.sin(theta), 0],
#                                       [-np.sin(theta),
#                                        np.cos(theta), 0], [0, 0, 1]])
#             block[:, :3] = np.dot(block_[:, :3], transform_mat)
#             map[:, :3] = np.dot(map_[:, :3], transform_mat)
            block[...] = block_[...]
            map[:, :3] = map_[:, :3]

            
        
        # parsing the original label to the dynamic label
        block[:, 3] = (block_[:, 3] != 0)
        map[:, 3:] = (map_[:, 3:] != 0 )
        
        print("radius; transformation; map points len: ", np.shape(map[:]))
        print("radius; transformation; scan points len: ", np.shape(block))
        

        
        ####################jupyter lab########################
        
#         # self-supervised dynamic label
#         center_x = np.mean(block[:, 0])
#         center_y = np.mean(block[:, 1])
#         # additional scan point
#         region = [center_x+2, center_x+3, center_y-2, center_y+3, 0, 2]
#         sspoints_ = np.mgrid[center_x+2:center_x+3:self.voxel_size, center_y-2:center_y+3:self.voxel_size, 0:2:self.voxel_size]
#         sspoints = sspoints_.reshape(3, -1).T
#         sslabels = np.ones([np.shape(sspoints)[0], 1],)
#         ssblock = np.concatenate((sspoints, sslabels), axis=1)
#         print("map before self-created point cloud shape: ", np.shape(map))
#         print("self-created point cloud shape: ", np.shape(ssblock))
#         map = np.concatenate((map, ssblock), axis=0)
#         print("map with self-created point cloud shape: ", np.shape(map))
                
        
        ####################jupyter lab########################


#         # get point and voxel in the format of sparse torch tensor
#         map_data = self.get_point_voxel(map[:, :3], map[:, 3], index)
#         scan_data = self.get_point_voxel(block[:, :3], block[:, 3], index)
        
        # get original point and voxel
#         map_data = None
#         scan_data = None
#         print(np.shape(map))
#         points_, labels_, proposals_ = map[:, :3], map[:, 3], map[:, 4]
#         pc_ = np.round(points_ / self.voxel_size).astype(np.int32)
#         pc_ -= pc_.min(0, keepdims=1)
        
#         _, inds, inverse_map = sparse_quantize(pc_,
#                                                return_index=True,
#                                                return_inverse=True)
        
#         if len(inds) > self.num_points:
#                 inds = np.random.choice(inds, self.num_points, replace=False)
    
#         pc = pc_[inds]
#         labels = labels_[inds]
        
#         if proposals_ is not None:
#             proposals = proposals_[inds]
#         else:
#             proposals = proposals_
        
        
#         return map_data, scan_data
#         map_data = self.get_original_point(map[:, :3], map[:, 3], map[:, 4], index)
#         scan_data = self.get_original_point(block[:, :3], block[:, 3], None, index)
        map_data = self.get_original_point(map[:], index)
        print("map_data points len: ", np.shape(map_data['pc']))
        scan_data = self.get_original_point(block, index)
        print("scan data points len: ", np.shape(scan_data['pc']))
        
        # neighbor_k ERASOR points
        if self.neighbor_k != 0:
            map_data['targets'] = self.propose_near_dynamic_points(map_data['pc'], map_data['targets'], k=self.neighbor_k)
        
        return map_data, scan_data

    @staticmethod
    def collate_fn(inputs):
        return sparse_collate_fn(inputs)
    
    def propose_near_dynamic_points(self, points, labels, leaf_size=40, k=10):
        """propose top k near points from dynamic points"""
        # nearest point search intialization
        tree = KDTree(points, leaf_size=leaf_size) 
        dist, ind = tree.query(points, k=k) 
        labels[np.unique(ind[labels==1])] = 1 #  index of the top 5 near points from dynamic points -> set -> label the point as dynamic point
        
        return labels
    
    
    def get_original_point(self, points, index):
        # points_ -> pc_ -> pc, labels_ -> labels, proposals_ -> proposals
        if np.shape(points)[-1] == 6: 
            points_, labels_, proposals_ = points[:, :3], points[:, 3], points[:, 4]
        elif np.shape(points)[-1] == 4:
            points_, labels_, proposals_ = points[:, :3], points[:, 3], None
        
        pc_ = np.round(points_ / self.voxel_size).astype(np.int32)
        pc_ -= pc_.min(0, keepdims=1)
        print("minimum pc: ", pc_.min(0, keepdims=1))
        
        _, inds, inverse_map = sparse_quantize(pc_,
                                               return_index=True,
                                               return_inverse=True)
        
        if len(inds) > self.num_points:
                inds = np.random.choice(inds, self.num_points, replace=False)
    
        pc = pc_[inds]
        labels = labels_[inds]
        
#         print("compare pc_ and pc: ", np.array_equal(pc_, pc)

                      
        if proposals_ is not None:
            proposals = proposals_[inds]
        else:
            proposals = proposals_        
        
        feed_dict = {
            'points_': points_,
            'pc_': pc_,
            'pc': pc,
            'targets_': labels_,
            'targets': labels,
            'proposals_': proposals_,
            'proposals': proposals,
            'file_name': self.scan_files[index],
            'inds': inds,
            'inverse_map': inverse_map,
        }
        return feed_dict
    
    
    def get_point_voxel(self, block, labels_, index):
        pc_ = np.round(block[:, :3] / self.voxel_size).astype(np.int32)
        pc_ -= pc_.min(0, keepdims=1)

        feat_ = block

        _, inds, inverse_map = sparse_quantize(pc_,
                                               return_index=True,
                                               return_inverse=True)

        if 'train' in self.split:
            if len(inds) > self.num_points:
                inds = np.random.choice(inds, self.num_points, replace=False)

        pc = pc_[inds]
        feat = feat_[inds]
        labels = labels_[inds]
        lidar = SparseTensor(feat, pc)
        labels = SparseTensor(labels, pc)
        labels_ = SparseTensor(labels_, pc_)
        inverse_map = SparseTensor(inverse_map, pc_)

        if self.visualize == False:
            feed_dict = {'lidar': lidar, 'targets': labels, 'targets_mapped': labels_, 'inverse_map': inverse_map, 'file_name': self.scan_files[index]}

        else:
            feed_dict = {'pc': block[:, :3], 'label': labels_, 'lidar': lidar, 'targets_mapped': labels_, 'inverse_map': inverse_map, 'file_name': self.scan_files[index]}

        return feed_dict

In [19]:
root = "/ws/data/erasor_carla/carla_v0.1"
voxel_size = 0.1
num_points = 180000
visualize = True
sample_stride = 1
split = "val"
proposal = None
submit = True
window = 10
save_npy = False
index = 230
radius = 50
neighbor_k = 0

In [20]:
dataset = ErasorCarlaInternal(root, voxel_size, num_points, visualize, split, proposal, submit, window=window, radius=radius, neighbor_k=neighbor_k,index=index )

In [21]:
map_data, scan_data = next(iter(dataset))

radius; concatenated scan:  (1165117, 4)
orignal; map point len:  (3597868,)
orignal; map point element:  [0. 2.]
odom:  [-8.8699127e+01  1.3943161e+02 -2.2663999e-02]
odom_:  [-8.8699127e+01  1.3943161e+02 -2.2663999e-02]
odom:  [-8.8699127e+01  1.3943161e+02 -2.2663999e-02]
radius; map points len:  (178205, 4)
radius; map point element:  [0. 2.]
radius; transformation; map points len:  (178205, 4)
radius; transformation; scan points len:  (1165117, 4)
minimum pc:  [[0 0 0]]
map_data points len:  (66070, 3)
minimum pc:  [[0 0 0]]
scan data points len:  (180000, 3)


In [22]:
def visualize_points(points, labels, label2color):
    points = (points - points.mean()) / points.std()
    colors = np.array([label2color[x] for x in labels])
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    visualizer = JVisualizer()
    visualizer.add_geometry(pcd)
    visualizer.show()

def save_points(points, labels, label2color, path):
#     points = (points - points.mean()) / points.std()
    colors = np.array([label2color[x] for x in labels])
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    o3d.io.write_point_cloud(path, pcd)


In [23]:
label2color = {0: (1.0, 0.0, 0.0), 1: (0.0, 0.0, 1.0), 2:(0.0, 0.0, 1.0)}
mpoints = map_data['pc']
mlabels = map_data['targets']
mproposals = map_data['proposals']

label2color = {0: (1.0, 0.0, 0.0), 1: (0.0, 0.0, 1.0), 2:(0.0, 0.0, 1.0)}
points = scan_data['pc']
labels = scan_data['targets']

In [24]:
path = "/ws/data/erasor_carla/debug/v0.1_map_radius.pcd"
save_points(mpoints, mlabels, label2color, path)

In [25]:
path = "/ws/data/erasor_carla/debug/v0.1_scan_radius.pcd"
save_points(points, labels, label2color, path)

In [46]:
visualize_points(mpoints, mlabels, label2color)

JVisualizer with 1 geometries

In [21]:
visualize_points(mpoints, mproposals, label2color)

JVisualizer with 1 geometries

In [10]:
# visualize_points(mpoints, mproposals, label2color)
save_points(mpoints, mproposals, label2color, "/ws/data/erasor_carla/debug/mpoints.pcd")

JVisualizer with 1 geometries

In [86]:
visualize_points(points, labels, label2color)

JVisualizer with 1 geometries

In [22]:
label2color = {0: (1.0, 0.0, 0.0), 1: (0.0, 0.0, 1.0), 2:(0.0, 0.0, 1.0)}
points = scan_data['pc']
labels = scan_data['targets']

In [23]:
points

array([[1174,  981,   14],
       [ 931, 1334,   14],
       [1039, 1029,   12],
       ...,
       [1136, 1021,   12],
       [1091, 1172,   14],
       [1280, 1127,   14]], dtype=int32)

In [24]:
mpoints

array([[   0, 2823,    0],
       [   0, 2824,    0],
       [   0, 2825,    0],
       ...,
       [ 499,   78,   56],
       [ 499,   81,   56],
       [ 499,   82,   56]], dtype=int32)

In [13]:
visualize_points(points, labels, label2color)

326.93422407407405

In [9]:
mpoints = map_data['pc']
mlabels = map_data['targets']

In [10]:
center_x = np.mean(mpoints[:, 0])
center_y = np.mean(mpoints[:, 1])
region = [center_x+2, center_x+3, center_y-2, center_y+3, 0, 2]
mlabels.shape

(180000,)

In [11]:
mpoints.shape

(180000, 3)

In [12]:
x_range = (region[0] < mpoints[:, 0]) & (region[1] > mpoints[:, 0])
y_range = (region[2] < mpoints[:, 1]) & (region[3] > mpoints[:, 1])
z_range = (region[4] < mpoints[:, 2]) & (region[5] > mpoints[:, 2])
# mpoints_region_idx = np.where((region[0] < mpoints[:, 0]) & (region[1] > mpoints[:, 0]) & (region[2] < mpoints[:, 1]) & (region[3] > mpoints[:, 1]) & (region[4] < mpoints[:, 2]) & (region[5] > mpoints[:, 2]))
mpoints_region_idx = np.where(x_range)

In [13]:
np.shape(mpoints_region_idx)

(1, 254)

In [14]:
x_range

array([False, False, False, ..., False, False, False])

In [15]:
mpoints[x_range & y_range & z_range]

array([], shape=(0, 3), dtype=int32)

In [16]:
static_mpoints = mpoints[(mlabels==0) & (mpoints[:, 2] > 20)]
random_point = np.random.randint(0, np.shape(static_mpoints)[0])
static_mpoints[random_point]

array([851, 531,  37], dtype=int32)

In [17]:
static_mpoints

array([[387, 627,  37],
       [242, 469,  23],
       [182, 461,  35],
       ...,
       [384, 584,  40],
       [367, 538,  30],
       [258, 231,  39]], dtype=int32)

In [18]:
points = scan_data['pc']
labels = scan_data['targets']

In [19]:
def visualize_points(points, labels, label2color):
    points = (points - points.mean()) / points.std()
    colors = np.array([label2color[x] for x in labels])
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    visualizer = JVisualizer()
    visualizer.add_geometry(pcd)
    visualizer.show()

def save_points(points, labels, label2color, path):
    points = (points - points.mean()) / points.std()
    colors = np.array([label2color[x] for x in labels])
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    o3d.io.write_point_cloud(path, pcd)

In [20]:
label2color = {0: (1.0, 0.0, 0.0), 1: (0.0, 0.0, 1.0), 2:(0.0, 0.0, 1.0)}

In [21]:
mlabels[mlabels==1].shape

(18859,)

In [22]:
# visualize_points(mpoints, mlabels, label2color)

In [89]:
path = "/ws/data/erasor_carla/carla_dataset/figure/v0.2/map_index{}_w10_enlarge_gt_ss.pcd".format(index)
save_points(mpoints, mlabels, label2color, path)

In [46]:
path = "/ws/data/erasor_carla/carla_dataset/figure/v0.2/map_index{}_w10_gt.pcd".format(index)
save_points(mpoints, mlabels, label2color, path)

In [47]:

tree = KDTree(mpoints, leaf_size=40) # nearest point search intialization
dist, ind = tree.query(mpoints, k=5) # search top 5 near points and return distance and index
mlabels[np.unique(ind[mlabels==1])] = 1 #  index of the top 5 near points from dynamic points -> set -> label the point as dynamic point

In [48]:
mlabels[mlabels==1].shape

(17620,)

In [49]:
path = "/ws/data/erasor_carla/carla_dataset/figure/v0.2/map_index{}_w10_enlarge_gt.pcd".format(index)
save_points(mpoints, mlabels, label2color, path)

In [ ]:
# visualize_points(mpoints, mlabels, label2color)

In [179]:
visualize_points(mpoints, mlabels, label2color)

JVisualizer with 1 geometries

In [72]:
points_ = scan_data['pc']
labels = scan_data['targets']
orig_points = scan_data['pc'][scan_data['inverse_map']]
orig_labels = scan_data['targets'][scan_data['inverse_map']]

In [73]:
points = (points_ - points_.mean()) / points_.std()

In [74]:
labels.shape

(183949,)

In [75]:
label2color = {0: (1.0, 0.0, 0.0), 1: (0.0, 0.0, 1.0), 2:(0.0, 0.0, 1.0)}
colors = np.array([label2color[x] for x in labels])

In [76]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)
visualizer = JVisualizer()
visualizer.add_geometry(pcd)
visualizer.show()

JVisualizer with 1 geometries

In [77]:
np.shape(points)

(183949, 3)

In [78]:
mpoints_ = map_data['pc']
mlabels = map_data['targets']
# orig_points = map_data['pc'][map_data['inverse_map']]
# orig_labels = map_data['targets'][map_data['inverse_map']]

In [79]:
mpoints = (mpoints_ - mpoints_.mean()) / mpoints_.std()

In [80]:
label2color = {0: (1.0, 0.0, 0.0), 1: (0.0, 0.0, 1.0), 2:(0.0, 0.0, 1.0)}
mcolors = np.array([label2color[x] for x in mlabels])

In [81]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(mpoints)
pcd.colors = o3d.utility.Vector3dVector(mcolors)
visualizer = JVisualizer()
visualizer.add_geometry(pcd)
visualizer.show()

JVisualizer with 1 geometries

In [ ]:
tree = KDTree(mpoints, leaf_size=40) # nearest point search intialization
dist, ind = tree.query(mpoints, k=5) # search top 5 near points and return distance and index
mlabels[np.unique(ind[mlabels==1])] = 1 #  index of the top 5 near points from dynamic points -> set -> label the point as dynamic point

In [82]:
np.shape(mpoints)

(240000, 3)

In [90]:
sspoints_ = np.mgrid[1:3:0.2, 2:6:0.2, 0:2:0.2]
sspoints = sspoints_.reshape(3, -1).T
sslabels = np.ones([np.shape(sspoints)[0], 1])

In [44]:
points

array([[-1.19546345,  0.59915196, -1.19161234],
       [-1.18391013,  0.59530085, -1.19161234],
       [-1.17235681,  0.58759864, -1.19161234],
       ...,
       [ 1.97399681, -0.34822012, -1.01831257],
       [ 1.99710345, -0.29045353, -1.01831257],
       [ 1.99710345, -0.28660243, -1.01831257]])

In [183]:
np.shape(sspoints)

(2000, 3)

In [184]:
np.shape(sslabels)

(2000, 1)

In [185]:
ssblock = np.concatenate((sspoints, sslabels), axis=1)

In [162]:
points = np.concatenate((points, sspoints), axis=0)

In [164]:
points

array([[-1.18889817,  0.44287013, -1.18523127],
       [-1.18889817,  0.45387082, -1.18523127],
       [-1.18523127,  0.39886739, -1.18523127],
       ...,
       [ 5.8       ,  4.8       ,  1.4       ],
       [ 5.8       ,  4.8       ,  1.6       ],
       [ 5.8       ,  4.8       ,  1.8       ]])